In [1]:
import pyterrier as pt
from pyterrier.measures import AP, nDCG, P, RR
import pandas as pd

if not pt.started():
    pt.init(tqdm="notebook")

PyTerrier 0.10.0 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
bm25_pl = (
    pt.rewrite.tokenise()
    >> pt.BatchRetrieve.from_dataset(
        "msmarcov2_passage", "terrier_stemmed", wmodel="BM25"
    )
    % 1000
)

12:10:10.537 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading lookup file directly from disk (SLOW) - try index.meta.index-source=fileinmem in the index properties file. 1 GiB of memory would be required.
12:10:10.543 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 8,6 GiB of memory would be required.


In [3]:
intents = pd.read_csv(
    "../../data/intent.tsv",
    delimiter="\t",
    header=None,
    names=("iid", "intent"),
    dtype=str,
)
qid_intent_mapping = pd.read_csv(
    "../../data/qid_iid_qrel.txt",
    delim_whitespace=True,
    header=None,
    names=["qid", "iid", "psg", "label"],
    dtype=str,
)
queries = pd.read_csv(
    "../../data/query.tsv",
    delimiter="\t",
    names=("qid", "query"),
    header=None,
    dtype=str,
)

/var/folders/yj/48233x_x22z9qclg68tnd_3r0000gn/T/ipykernel_54023/1118679602.py:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  qid_intent_mapping = pd.read_csv(


In [4]:
METRICS = [AP @ 100, nDCG @ 10, P @ 10, RR]

**Experiment 1**: BM25 using intents as queries and evaluating with intent QRels


In [5]:
pt.Experiment(
    [bm25_pl],
    intents.rename(columns={"iid": "qid", "intent": "query"}),
    pt.io.read_qrels("../../data/qrels.txt"),
    METRICS,
    names=["BM25 (intents as queries)"],
    save_dir="runs",
)

,name,AP@100,nDCG@10,P@10,RR
0,BM25 (intents as queries),0.057782,0.116401,0.110145,0.262117


Same experiment here, but reporting results per intent:


In [6]:
pt.Experiment(
    [bm25_pl],
    intents.rename(columns={"iid": "qid", "intent": "query"}),
    pt.io.read_qrels("../../data/qrels.txt"),
    METRICS,
    names=["BM25 (intents as queries)"],
    save_dir="runs",
    perquery=True,
)

,name,qid,measure,value
0,BM25 (intents as queries),1,RR,1.000000
1,BM25 (intents as queries),1,P@10,0.200000
2,BM25 (intents as queries),1,nDCG@10,0.275553
3,BM25 (intents as queries),1,AP@100,0.208333
36,BM25 (intents as queries),10,RR,0.058824
...,...,...,...,...
31,BM25 (intents as queries),8,AP@100,0.000000
32,BM25 (intents as queries),9,RR,0.500000
33,BM25 (intents as queries),9,P@10,0.600000
34,BM25 (intents as queries),9,nDCG@10,0.576669


**Experiment 2**: BM25 using original queries and evaluating with intent QRels


In [7]:
# here: "qid" column holds intent IDs, "query" column holds corresponding original queries
intents_with_orig_queries = (
    intents.merge(qid_intent_mapping, on="iid")
    .merge(queries, on="qid")[["iid", "query"]]
    .rename(columns={"iid": "qid"})
).drop_duplicates()

In [8]:
pt.Experiment(
    [bm25_pl],
    intents_with_orig_queries,
    pt.io.read_qrels("../../data/qrels.txt"),
    METRICS,
    names=["BM25 (intents with original queries)"],
    save_dir="runs",
)

,name,AP@100,nDCG@10,P@10,RR
0,BM25 (intents with original queries),0.048964,0.07325,0.081159,0.203476


Same experiment here, but reporting results per intent (original query):


In [9]:
pt.Experiment(
    [bm25_pl],
    intents_with_orig_queries,
    pt.io.read_qrels("../../data/qrels.txt"),
    METRICS,
    names=["BM25 (intents with original queries)"],
    save_dir="runs",
    perquery=True,
)

,name,qid,measure,value
0,BM25 (intents with original queries),1,RR,1.000000
1,BM25 (intents with original queries),1,P@10,0.200000
2,BM25 (intents with original queries),1,nDCG@10,0.220726
3,BM25 (intents with original queries),1,AP@100,0.152778
36,BM25 (intents with original queries),10,RR,0.062500
...,...,...,...,...
31,BM25 (intents with original queries),8,AP@100,0.002233
32,BM25 (intents with original queries),9,RR,0.003040
33,BM25 (intents with original queries),9,P@10,0.000000
34,BM25 (intents with original queries),9,nDCG@10,0.000000
